<a href="https://colab.research.google.com/github/RafaelTorresCH/senalesysistemas_/blob/main/PARCIAL_2/P2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instala todas las librerías requeridas
!pip install streamlit pyngrok -q
!pip install soundfile -q
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 55.7 MB/s eta 0:00:00


In [2]:
%%writefile app.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import hilbert, butter, lfilter, freqz, tf2zpk
import librosa
import yt_dlp
import os

# --- CONFIGURACIÓN DE LA PÁGINA ---
st.set_page_config(
    page_title="Dashboard SSB-AM",
    page_icon="📡",
    layout="wide"
)

# --- FUNCIONES AUXILIARES ---

def rect(t, duracion):
    return np.where(np.abs(t) <= duracion / 2, 1, 0)

@st.cache_data
def generar_pulso_rectangular(fs, duracion_total, duracion_pulso, amplitud):
    t = np.arange(0, duracion_total, 1/fs)
    t_centrado = t - duracion_total / 2
    pulso = amplitud * rect(t_centrado, duracion_pulso)
    return t, pulso

@st.cache_data
def descargar_y_cargar_audio(url, cookies_path=None):
    audio_file = 'downloaded_audio.wav'
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
        'outtmpl': audio_file.replace('.wav', ''),
        'quiet': True,
        'overwrite': True,
    }
    if cookies_path:
        ydl_opts['cookiefile'] = cookies_path

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        x, fs = librosa.load(audio_file, sr=None, mono=True)
        os.remove(audio_file)
        return x, fs
    except Exception as e:
        st.error(f"Error al procesar el audio: {e}")
        return None, None

def plot_signal(t, signal, title, xlabel='Tiempo [s]', ylabel='Amplitud'):
    fig, ax = plt.subplots(figsize=(10, 3))
    ax.plot(t, signal)
    ax.set_title(title, fontsize=14)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.grid(True)
    st.pyplot(fig)

def plot_spectrum(signal, fs, title, xlim=None):
    N = len(signal)
    f = np.fft.fftshift(np.fft.fftfreq(N, 1/fs))
    Y = np.fft.fftshift(np.fft.fft(signal))
    fig, ax = plt.subplots(figsize=(10, 3))
    ax.plot(f, np.abs(Y))
    ax.set_title(title, fontsize=14)
    ax.set_xlabel('Frecuencia [Hz]')
    ax.set_ylabel('|M(f)|')
    ax.grid(True)
    if xlim:
        ax.set_xlim(xlim)
    st.pyplot(fig)

def plot_bode(b, a, fs, title="Diagrama de Bode del Filtro"):
    w, h = freqz(b, a, worN=8000, fs=fs)
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 5))
    fig.suptitle(title, fontsize=14)
    ax1.plot(w, 20 * np.log10(abs(h)))
    ax1.set_ylabel('Magnitud [dB]')
    ax1.grid(True)
    ax2.plot(w, np.angle(h, deg=True))
    ax2.set_xlabel('Frecuencia [Hz]')
    ax2.set_ylabel('Fase [grados]')
    ax2.grid(True)
    st.pyplot(fig)

def plot_pole_zero(b, a, title="Plano de Polos y Ceros"):
    zeros, poles, k = tf2zpk(b, a)
    fig, ax = plt.subplots(figsize=(6, 6))
    unit_circle = plt.Circle((0, 0), 1, color='gray', fill=False, linestyle='--')
    ax.add_artist(unit_circle)
    ax.plot(np.real(zeros), np.imag(zeros), 'o', markersize=10, label='Ceros')
    ax.plot(np.real(poles), np.imag(poles), 'x', markersize=10, label='Polos')
    ax.set_title(title, fontsize=14)
    ax.set_xlabel('Parte Real')
    ax.set_ylabel('Parte Imaginaria')
    ax.grid(True)
    ax.axis('equal')
    ax.legend()
    st.pyplot(fig)

# --- INTERFAZ DE STREAMLIT ---
st.title("📡 Dashboard de Modulación y Demodulación SSB-AM")
st.markdown("Este dashboard interactivo modela el proceso de modulación SSB-AM usando el método de fase.")

# --- BARRA LATERAL DE CONTROLES ---
st.sidebar.title("Configuración")
st.sidebar.markdown("**Nombres:** Rafael Ricardo Torres Choperena")

tipo_senal = st.sidebar.selectbox(
    "1. Seleccione la señal mensaje",
    ("Señal Pulso Rectangular", "Segmento de Canción (YouTube)")
)
fc = st.sidebar.slider("2. Frecuencia de la portadora (Fc)", 1000, 10000, 5000, 100)
tipo_banda = st.sidebar.radio(
    "3. Seleccione la banda lateral",
    ("Superior (USB)", "Inferior (LSB)"),
    index=0
)

# OPCIÓN PARA SUBIR COOKIES DESDE STREAMLIT
st.sidebar.markdown("---")
st.sidebar.subheader("Opcional: Cookies de YouTube")
cookies_file = st.sidebar.file_uploader(
    "Sube tu archivo de cookies (formato Netscape)",
    type=["txt", "cookies"]
)

cookies_path = None
if cookies_file:
    cookies_path = "cookies_uploaded.txt"
    with open(cookies_path, "wb") as f:
        f.write(cookies_file.read())
    st.sidebar.success("✅ Cookies cargadas correctamente.")

# --- LÓGICA DE GENERACIÓN DE SEÑAL ---
m = None
t = None
fs = 20 * fc

if tipo_senal == "Señal Pulso Rectangular":
    t, m = generar_pulso_rectangular(fs, 1.0, 0.1, 1.0)
else:
    url = st.sidebar.text_input("Ingrese la URL de YouTube", "https://youtu.be/nKX8ZNzuvNs?si=eNeFqEJlBZeoLt4o")
    if url:
        with st.spinner("Descargando y procesando audio..."):
            full_audio, fs_audio = descargar_y_cargar_audio(url, cookies_path=cookies_path)

            if full_audio is not None:
                fs = fs_audio
                duracion_total = len(full_audio) / fs
                num_segmentos = int(duracion_total / 5)

                if num_segmentos < 1:
                    st.warning("El audio es demasiado corto (menos de 5 segundos).")
                    m = None
                else:
                    st.sidebar.info(f"Audio cargado. {num_segmentos} segmentos de 5 segundos encontrados.")
                    segmento_elegido = st.sidebar.slider(
                        "4. Seleccione el segmento de audio a procesar",
                        1,
                        num_segmentos,
                        1
                    )
                    start_sample = int((segmento_elegido - 1) * 5 * fs)
                    end_sample = start_sample + int(5 * fs)
                    m = full_audio[start_sample:end_sample]
                    t = np.arange(len(m)) / fs

                    if fc > fs / 2:
                        st.warning(f"Fc ({fc} Hz) es muy alta para la Fs del audio ({fs} Hz). Ajustando Fc a {fs/4} Hz.")
                        fc = fs / 4

# --- CUERPO PRINCIPAL DEL DASHBOARD ---
if m is not None:
    st.markdown("---")
    st.header("1. Señal Mensaje (Banda Base)")
    plot_signal(t, m, f"Señal Mensaje en el Tiempo, m(t) ({tipo_senal})")
    plot_spectrum(m, fs, "Espectro de la Señal Mensaje", xlim=[-fc, fc])

    st.markdown("---")
    st.header("2. Proceso de Modulación SSB-AM")
    if tipo_banda == "Superior (USB)":
        st.latex("s_{SSB}(t) = m(t) \\cos(2\\pi f_c t) - \\hat{m}(t) \\sin(2\\pi f_c t)")
    else:
        st.latex("s_{SSB}(t) = m(t) \\cos(2\\pi f_c t) + \\hat{m}(t) \\sin(2\\pi f_c t)")

    m_h = np.imag(hilbert(m))
    portadora_cos = np.cos(2 * np.pi * fc * t)
    portadora_sin = np.sin(2 * np.pi * fc * t)

    if tipo_banda == "Superior (USB)":
        s_ssb = m * portadora_cos - m_h * portadora_sin
    else:
        s_ssb = m * portadora_cos + m_h * portadora_sin

    plot_signal(t, s_ssb, "Señal Modulada SSB en el Tiempo")
    plot_spectrum(s_ssb, fs, f"Espectro de la Señal Modulada ({tipo_banda})", xlim=[0, fc * 2])

    st.markdown("---")
    st.header("3. Proceso de Demodulación SSB-AM")
    st.latex("v(t) = s_{SSB}(t) \\cdot \\cos(2\\pi f_c t) \\implies m_{rec}(t) = \\text{LPF}\\{v(t)\\}")
## proceso de filtrado
    v = s_ssb * portadora_cos
    orden = 4
    f_corte = fc * 0.5
    b, a = butter(orden, f_corte, btype='low', analog=False, fs=fs)
    m_recuperada = lfilter(b, a, v)

    st.subheader("Filtro IIR Pasa-Bajas Requerido")
    col1, col2 = st.columns(2)
    with col1:
        plot_bode(b, a, fs)
    with col2:
        plot_pole_zero(b, a)

    st.subheader("Señal Recuperada")
    plot_signal(t, m_recuperada, "Señal Recuperada en el Tiempo")
    plot_spectrum(m_recuperada, fs, "Espectro de la Señal Recuperada", xlim=[-fc, fc])

    if tipo_senal != "Señal Pulso Rectangular":
        st.subheader("Comparación de Audio")
        st.audio(m, sample_rate=int(fs))
        st.audio(m_recuperada, sample_rate=int(fs))
else:
    st.info("Sube el archivo de cookies si necesitas desbloquear contenido restringido y configura los parámetros en la barra lateral para comenzar la simulación.")


Writing app.py


In [3]:
# Reemplaza "TU_AUTHTOKEN" con tu token real de ngrok
# Consíguelo gratis en: https://dashboard.ngrok.com/get-started/your-authtoken
authtoken = "2zZAkBm8Fh665AIQmAbAovQCN8P_2Hso7sscNP55vLTVCAikM"

# --- No modifiques el código debajo de esta línea ---

# Ejecuta la app de Streamlit en segundo plano en el puerto 8501
!nohup streamlit run app.py --server.port 8501 &

# Importa pyngrok, configura el token y crea el túnel
from pyngrok import ngrok
import time

try:
    ngrok.set_auth_token(authtoken)
    public_url = ngrok.connect(8501)
    time.sleep(2) # Pequeña pausa para asegurar que el túnel se establezca
    print(f"✅ ¡Dashboard listo! Accede aquí: {public_url}")
except Exception as e:
    print(f"❌ Error al iniciar ngrok: {e}")
    print("Asegúrate de haber reemplazado 'TU_AUTHTOKEN' con tu token real.")

nohup: appending output to 'nohup.out'
✅ ¡Dashboard listo! Accede aquí: NgrokTunnel: "https://f1659a7a7da1.ngrok-free.app" -> "http://localhost:8501"
